## RL-Umgebungen

In [1]:
# HIDDEN

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

#### Was ist eine Environment?

- Eine Environment kann sein:
  - ein Spiel, wie ein Videospiel.
  - eine Simulation eines realen Szenarios, z. B. eines Roboters, des Nutzerverhaltens oder der Börse
  - jede andere Environment mit einem _Agenten_, der _Aktionen_ durchführt, _Beobachtungen_ macht und _Belohnungen_ erhält
  
Eine Anmerkung zur Terminologie: Wir werden _Agent_ und _Spieler_ synonym verwenden 

#### Laufendes Beispiel: Gefrorener See

Als laufendes Beispiel für eine Environment verwenden wir die [Frozen Lake](https://www.gymlibrary.ml/environments/toy_text/frozen_lake/) Environment aus [OpenAI Gym](https://www.gymlibrary.ml/), die die Standardschnittstelle für RL-Probleme bietet. Wir können die Environment wie folgt visualisieren:

In [2]:
import gym
env = gym.make("FrozenLake-v1", is_slippery=False)

In [3]:
# HIDDEN
from utils_01 import fix_frozen_lake_render
fix_frozen_lake_render(env)

In [4]:
env.reset()
env.render()


P...
.O.O
...O
O..G


Das Ziel ist es, dass der Spieler (P) das Ziel (G) erreicht, indem er auf den gefrorenen Seesegmenten (O) läuft, ohne in die Löcher (O) zu fallen.

**Hinweis**: Das Rendering der Environment wurde vom OpenAI Gym Rendering abgewandelt, damit es in dieser interaktiven Lernplattform deutlich dargestellt werden kann.

Anmerkungen:

Der Code für das Ersetzen des Renderings wird nicht angezeigt.

#### Bewegung

Der Spieler kann sich auf dem gefrorenen See bewegen. Zum Beispiel:

In [5]:
env.step(1) # 1 -> Down
env.render()

  (Down)
....
PO.O
...O
O..G


Mach dir vorerst keine Gedanken über "Schritt(1)"; dazu kommen wir noch 

Du kannst sehen, dass der Spieler (P) sich nach unten bewegt hat.

#### Ziel

Spule viele Schritte vorwärts, und du hast das Puzzle vervollständigt:

In [6]:
env.step(1)
env.step(2)
env.step(1)
env.step(2)
env.step(1)
env.step(2)
env.render()

  (Right)
....
.O.O
...O
O..P


Du hast das Ziel erreicht, indem du die untere rechte Ecke erreicht hast.

#### Was macht eine Environment aus?

Eine Environment besteht aus mehreren Schlüsselkomponenten, die wir auf den folgenden Folien erläutern werden.

#### Staaten

- Wir verwenden den Begriff _Zustand_ informell für alles, was die Environment betrifft 

In [7]:
env.reset()
env.render()


P...
.O.O
...O
O..G


- Das ist zum Beispiel der Ausgangszustand der Environment.
- Der Spieler befindet sich oben links, in der Nähe gibt es gefrorenes Eis usw.
- Wir werden das Konzept des Zustands verwenden, um über unsere Environment zu sprechen, aber es wird nicht in der "API" erscheinen.

#### Aktionen

- Hier kann der Spieler zwischen 4 möglichen Aktionen wählen: links, unten, rechts, oben
- Der Raum aller möglichen Aktionen wird **Aktionsraum** genannt.
- In SL unterscheiden wir zwischen Regression (kontinuierlich $y$) und Klassifikation (kategorisch $y$)
- Auch im RL kann der Aktionsraum kontinuierlich oder diskret sein
- In diesem Fall ist er diskret (4 Möglichkeiten)
- Der Code stimmt überein:

In [8]:
env.action_space

Discrete(4)

#### Beobachtungen

- Die Beobachtungen sind die _Teile des Zustands, die der Agent sehen kann_.
- Manchmal kann der Agent alles sehen; wir nennen das _voll beobachtbar_.
- Oft gibt es aber auch _teilweise beobachtbare_ Umgebungen
- Im Beispiel des gefrorenen Sees kann der Agent nur seinen eigenen Standort von den 16 Feldern sehen.
- Dem Agenten wird nicht durch direkte Beobachtungen "gesagt", wo die Löcher sind, also muss er dies durch Versuch und Irrtum _erlernen_.

#### Beobachtungen

- Der Raum aller möglichen Beobachtungen wird **Beobachtungsraum** genannt.
- Du kannst dir den Aktionsraum als Analogon zum Ziel beim überwachten Lernen vorstellen.
- Du kannst dir den Beobachtungsraum analog zu den Merkmalen beim überwachten Lernen vorstellen.


Hier haben wir einen diskreten Beobachtungsraum, der aus den 16 möglichen Spielerpositionen besteht:

In [9]:
env.observation_space

Discrete(16)

#### Belohnungen

- Beim überwachten Lernen besteht das Ziel in der Regel darin, gute Vorhersagen zu treffen.
- Je nach Ziel kannst du verschiedene Verlustfunktionen ausprobieren, aber das allgemeine Konzept ist dasselbe.
- Im RL kann das Ziel alles sein.
- Aber genau wie bei SL musst du etwas _optimieren_.
- Im RL versuchen wir, die **Belohnung** zu maximieren.

#### Belohnungen 

Im Beispiel des Gefrorenen Sees erhält der Agent eine Belohnung, wenn er das Ziel erreicht.

In [10]:
env.reset()
obs, reward, done, _ = env.step(0)
env.render()
print("reward =", reward)

  (Left)
P...
.O.O
...O
O..G
reward = 0.0


In [11]:
obs, reward, done, _ = env.step(1)
print("reward =", reward)

reward = 0.0


Immer noch keine Belohnung, lass uns weitermachen...

#### Belohnungen

In [12]:
env.step(1)
env.step(2)
env.step(1)
env.step(2)
env.step(1)
obs, reward, done, _ = env.step(2)
env.render()
print("reward =", reward)

  (Right)
....
.O.O
...O
O..P
reward = 1.0


Wir haben eine Belohnung von 1,0 für das Erreichen des Ziels erhalten.

#### Environment-Agent-Schleife

Beachte, wie der Agent (in diesem Fall wir) und die Environment in einer Hin- und Her-Schleife kommunizieren:

![](img/RL-loop.png)

Das ist das klassische Diagramm, das du überall siehst.

Das Ziel ist es, das Verhalten des Agenten automatisch zu lernen (bleib dran!).

#### Aktionen repräsentieren

- Um RL-Software zu verwenden, brauchen wir eine numerische Darstellung unseres Aktionsraums und unseres Beobachtungsraums.
- In diesem Fall haben wir 4 mögliche diskrete Aktionen, die wir als {0,1,2,3} für (links, unten, rechts, oben) kodieren können.
- Aus diesem Grund haben wir früher

In [13]:
env.step(1)

nach unten zu gehen.

#### Darstellung von Beobachtungen

- Wir brauchen auch eine numerische Darstellung unserer Beobachtungen.
- Hier gibt es 16 mögliche Positionen des Spielers. Diese werden wie folgt von 0-15 kodiert:

```
 0 1 2 3
 4 5 6 7
 8 9 10 11
12 13 14 15
```

Diese Details zur Frozen Lake-Environment findest du auch in der [Dokumentation](https://www.gymlibrary.ml/environments/toy_text/frozen_lake/).

#### Darstellung von Beobachtungen

Am Anfang sehen wir "0", weil wir oben links beginnen:

In [14]:
env.reset()
env.render()


P...
.O.O
...O
O..G


Nachdem wir uns nach unten bewegt haben (Aktion 1), gehen wir auf Position 4.

In [15]:
obs, reward, done, _ = env.step(1)
obs

4

Die Beobachtung wird von der Methode `step()` zurückgegeben.

#### Nicht-deterministische Umgebungen

- Bisher hat die Ausführung einer bestimmten Aktion in einem bestimmten Zustand immer zum gleichen neuen Zustand geführt.
- Mit anderen Worten: Unsere Frozen Lake-Environment war _deterministisch_.
- Manche Umgebungen sind _nicht-deterministisch_, das heißt, das Ergebnis einer Aktion kann zufällig sein.
- Wir können einen nicht-deterministischen Frozen Lake wie folgt initialisieren:

In [16]:
env_slippery = gym.make("FrozenLake-v1", is_slippery=True)

In [17]:
# HIDDEN
env_slippery.seed(4)
fix_frozen_lake_render(env_slippery)

In [18]:
env_slippery.reset()
env_slippery.render()


P...
.O.O
...O
O..G


#### Nicht-deterministische Umgebungen

In [19]:
env_slippery.step(1) # move down
env_slippery.render()

  (Down)
....
PO.O
...O
O..G


Der Umzug nach unten hat nicht wie geplant funktioniert.

In [20]:
env_slippery.step(1) # move down
env_slippery.render()

  (Down)
....
.P.O
...O
O..G


Diesmal hat es geklappt.

In dieser "glitschigen" Frozen Lake-Environment funktioniert die Bewegung nur in 1/3 der Fälle wie vorgesehen.

#### Episoden

- Das Spielen des Gefrorenen Sees hat ein Ende - entweder du fällst in ein Loch oder du erreichst das Ziel.
- Ein Durchlauf reicht jedoch nicht aus, damit ein RL-Algorithmus daraus lernen kann.
- Er braucht mehrere Durchläufe, die **Episode** genannt werden.
- Nach einer Episode wird die Environment zurückgesetzt.

#### Episoden

Die Methode `step()` gibt ein Flag zurück, das uns sagt, ob die Episode vorbei ist:

In [21]:
obs, reward, done, _ = env_slippery.step(1)
done

True

In [22]:
env_slippery.render()

  (Down)
....
.P.O
...O
O..G


Hier ist die Episode zu Ende, weil wir in ein Loch gefallen sind.

#### Episoden

- In einigen Umgebungen (z. B. Frozen Lake) erhältst du die Belohnungen erst am Ende einer Episode.
- In anderen Umgebungen können Belohnungen in jedem **Zeitschritt** (d.h. nach einer Aktion) erhalten werden.

#### Alles zusammenfügen

- Wir haben jetzt über die wichtigsten Komponenten einer RL-Environment gesprochen:
  - Zustände
  - Handlungen
  - Beobachtungen
  - Belohnungen
  - Episoden
  

#### SL-Datensätze vs. RL-Umgebungen

- Beim überwachten Lernen erhältst du normalerweise einen Datensatz.
- Beim RL fungiert die Environment als _Datengenerator_.
  - Je öfter du die Environment durchspielst, desto mehr "Daten" generierst du und desto mehr kannst du lernen.
- Man kann RL auch mit einem vorher gesammelten Datensatz durchführen (das nennt man _offline RL_), aber das ist für uns nicht relevant.

#### Lass uns das Gelernte anwenden!

## Selbstfahrendes Auto Environment
<!-- multiple choice -->

Du benutzt RL, um ein selbstfahrendes Auto zu trainieren. Die Auto-KI nutzt verschiedene Kameras und Sensoren als Input und muss den Winkel des Lenkrads sowie den Winkel der Gas- und Bremspedale am Boden bestimmen.

#### Ist der Beobachtungsraum kontinuierlich oder diskret?

- [x] Kontinuierlich
- [ ] Diskret | In diesem Fall sind die Beobachtungen die Sensoreingaben, z. B. Tiefenschätzungen.

#### Ist der Aktionsraum kontinuierlich oder diskret?

- [x] Kontinuierlich
- [ ] Diskret | Die Aktionen sind Winkel; sie stammen nicht aus einer diskreten Menge von Optionen.

#### Was wäre die sinnvollste Belohnungsstruktur für diese Environment?

- [ ] Die Belohnung entspricht der Zeit, die das Auto fahren konnte, ohne zu verunglücken | Wie hoch wäre die Belohnung, wenn sich das Auto nie bewegt?
- [x] Die Belohnung entspricht der Strecke, die das Auto fahren konnte, ohne zu verunglücken | Ja, das klingt gut!
- [ ] +1 Belohnung wenn das Auto crashed | Erinnnere Dich daran, dass wir versuchen die Belohnung zu maximieren und nicht zu minimieren.

## Episoden vs. Zeitschritte
<!-- multiple choice -->

Fülle die Lücken im folgenden Satz aus 

*Beim Reinforcement Learning führt man wiederholt Aktionen aus, bis das \_\_\_\_\_ endet. Dabei kann es sich um nur ein \_\_\_\_\_ handeln oder um sehr viele.*

- [ ] Zeitschritt / Belohnung | Prüfe das erste Leerzeichen sorgfältig!
- [ ] Belohnung / Zeitschritt | Versuche es noch einmal!
- [ ] Zeitschritt / Episode | Versuche es noch einmal!
- [x] Folge / Zeitschritt | Du hast es geschafft!


## Gym's taxi environment
<!-- coding exercise -->

In dieser Übung schauen wir uns eine der textbasierten Umgebungen an, die mit
OpenAI Gym mitgelieferten textbasierten Umgebungen, der sogenannten Taxi-Environment.
Dokumentation [hier](https://www.gymlibrary.ml/environments/toy_text/taxi/).

In [33]:
# EXERCISE
import gym

taxi = gym.make("Taxi-v3")
obs = taxi.reset(seed=5)
taxi.render()

# Add calls to `step` here!

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In dieser Übung wird das Taxi durch die gelbe Markierung dargestellt,
das sich derzeit unten links auf dem Spielfeld befindet. Das ":" kann überquert werden, aber das "|" nicht.
Das Ziel ist es, Fahrgäste aufzunehmen und sie abzusetzen.

Es gibt 6 mögliche Aktionen:
runter (0), Hoch (1), Rechts (2), Links (3), Aufnehmen (4), Absetzen (5).

In [36]:
taxi.action_space

Discrete(6)

Der folgende Code gibt die Beobachtung in einem besser lesbaren Format aus:

In [37]:
print("Taxi row: %d\nTaxi col: %d\nPassenger loc: %d\nDestination loc: %d" % tuple(taxi.decode(obs)))

Taxi row: 4
Taxi col: 0
Passenger loc: 0
Destination loc: 2


Die möglichen Orte sind "R" (0), "G" (1), "Y" (2), "B" (3) und im Taxi (4). Der Fahrgast befindet sich also gerade in "R" und ist auf dem Weg nach "Y".

Um die folgende Frage zu beantworten, musst du den Code ändern, ihn ausführen und alle relevanten Ausgaben ausdrucken.

In [42]:
# SOLUTION
import gym

taxi = gym.make("Taxi-v3")
obs = taxi.reset(seed=5)
taxi.render()
taxi.step(1)
taxi.step(1)
taxi.step(1)
taxi.step(1)
taxi.render()
taxi.step(4)
taxi.step(0)
taxi.step(0)
taxi.step(0)
taxi.step(0)
taxi.render()
obs, reward, done, _ = taxi.step(5)
print(reward)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
20


In [27]:
# TODO: in this case there is no testing of the code; the code is only used to explore
# this may not work well with the framework - add an automated test to the code as well? 
# e.g. can check that the state of the env is correct?

#### Wie viel Belohnung erhält der Agent für das erfolgreiche Absetzen des Passagiers?

- [ ] 0 | Versuche, die notwendigen Aktionen mit `taxi.step()` auszuführen und die Belohnungen auszudrucken.
- [ ] 5 | Versuche, die notwendigen Aktionen mit `taxi.step()` auszuführen und die Belohnungen auszudrucken.
- [ ] 10 | Versuche, die notwendigen Aktionen mit `taxi.step()` auszuführen und die Belohnungen auszudrucken.
- [x] 20 | Du hast es geschafft!

## Beobachtungen vs. Renderings
<!-- coding exercise -->

Die Environment des gefrorenen Sees ermöglicht es uns, eine visuelle Darstellung der Environment zu erstellen, die wir bereits gesehen haben. Diese Darstellung dient der Fehlersuche und wird vom Agenten/Algorithmus _nicht_ gesehen. Deine Aufgabe ist es, eine bestimmte Frozen Lake-Environment zu spielen, **ohne auf die visuelle Darstellung zu achten** (kein Schummeln!). Fülle die Liste "Aktionen" so auf, dass sie eine Reihe von Aktionen enthält, die den Agenten korrekt zum Ziel bringen. Was du hier erlebst, ist das, was ein RL-Algorithmus beim Lernen "sieht"!

Beachte, dass die gegebene Frozen Lake Environment 3x3 statt 4x4 ist. Der Beobachtungsraum geht also von 0 bis 8 statt von 0 bis 15.

In [28]:
# EXERCISE

import gym
import numpy as np
np.random.seed(1)
env = gym.make("FrozenLake-v1", 
               desc=gym.envs.toy_text.frozen_lake.generate_random_map(size=3, p=0.3), 
               is_slippery=False)
env.render = None

obs = env.reset()
actions = ____
for action in actions:
    obs, reward, done, _ = env.step(action)
    print("Obs:", obs, "Reward:", reward, "Done:", done)


NameError: name '____' is not defined

In [6]:
# SOLUTION

import gym.envs.toy_text
import numpy as np
np.random.seed(1)
env = gym.make("FrozenLake-v1", desc=gym.envs.toy_text.frozen_lake.generate_random_map(size=3, p=0.3), is_slippery=False)
env.render = None

obs = env.reset()
actions = []
# BEGIN SOLUTION
actions = [1,2,1,2]
# END SOLUTION
for action in actions:
    obs, reward, done, _ = env.step(action)
    print("Obs:", obs, "Reward:", reward, "Done:", done)

Obs: 3 Reward: 0.0 Done: False
Obs: 4 Reward: 0.0 Done: False
Obs: 7 Reward: 0.0 Done: False
Obs: 8 Reward: 1.0 Done: True


In [9]:
# HIDDEN
import numpy as np
import gym.envs.toy_text
from utils_01 import fix_frozen_lake_render
np.random.seed(1)
env = gym.make("FrozenLake-v1", desc=gym.envs.toy_text.frozen_lake.generate_random_map(size=3, p=0.3), is_slippery=False)
env.reset()
fix_frozen_lake_render(env)
env.render()


PO.
...
O.G


#### Der beste Weg zum Ziel

Wenn man bedenkt, dass die Aktionen 0, 1, 2, 3 jeweils für links, unten, rechts und oben stehen, welche der folgenden Aussagen beschreibt den besten Weg zum Ziel richtig?

- [ ] Beginne mit der Bewegung nach unten, dann wieder nach unten
- [x] Bewege dich zuerst nach unten, dann nach rechts
- [ ] Erst nach rechts, dann wieder nach rechts
- [ ] Erst nach rechts, dann nach unten